# Modeling

I transformed the molecules using Color Refinement.

[Here is a demo of Color Refinement](https://github.com/parkerburchett/TDC-DeepLearning/blob/main/utils/ColorRefinement_tutorial.ipynb)

I then tested some simple MLP with different loss and optimizer functions.

I then transformed the bags of colors for each neighborhood (0-10) using PCA and trained a "out of the box" LGBM regressor model. That model had a AUPRC of .8375+- .0093. That is a ~.09 improvement on the model that TDC is currently using.

These are the outcomes TDC's current model.
https://tdcommons.ai/benchmark/admet_group/10cyp2c9i/


I ran this in Google Colab and then saved it to this repo.

In [1]:
!pip install pysmiles
!pip install pyTDC
!git clone https://github.com/parkerburchett/TDC-DeepLearning

fatal: destination path 'TDC-DeepLearning' already exists and is not an empty directory.


In [2]:
from google.colab import drive
drive.mount('/content/drive')
import networkx as nx
import tensorflow as tf
from sklearn.metrics import precision_recall_curve, auc
from tensorflow.keras.optimizers import SGD
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
import pickle
from pysmiles import read_smiles
from tdc.single_pred import ADME
import os
os.chdir('/content/TDC-DeepLearning/')
from utils import ColorRefinement, utils

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
color_graphs = pickle.load(open('/content/drive/MyDrive/SpringBoard/Therapeutic Data Commons Projects/data/color_graphs.pkl', 'rb'))

# This is the 0-10 hop dfs of each of the molecules. Since it takes 45 minutes to run I ran it once and am just loading it each time

In [4]:
color_graphs[0] # a single molecule

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,4960,4961,4962,4963,4964,4965,4966,4967,4968,4969,4970,4971,4972,4973,4974,4975,4976,4977,4978,4979,4980,4981,4982,4983,4984,4985,4986,4987,4988,4989,4990,4991,4992,4993,4994,4995,4996,4997,4998,4999
iteration_number,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
color_0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
color_1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
color_2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
color_3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
color_4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
color_5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
color_6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
color_7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
color_8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


All of the feautre vectors are very spare, typically 95% + zeros.

In [5]:
def get_n_dem_embedding(color_graphs, n):
  embeddings = []
  for g in color_graphs:
    embeddings.append(g.values[n,:])
  return pd.DataFrame(np.array(embeddings))


In [6]:
feature_dfs = [get_n_dem_embedding(color_graphs, n) for n in range(0,10)]

In [7]:
feature_dfs[0]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,4960,4961,4962,4963,4964,4965,4966,4967,4968,4969,4970,4971,4972,4973,4974,4975,4976,4977,4978,4979,4980,4981,4982,4983,4984,4985,4986,4987,4988,4989,4990,4991,4992,4993,4994,4995,4996,4997,4998,4999
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12660,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12661,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12662,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12663,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Each row is a molecule, and each colum is a count of the final colors of each of it's nodes.

In [13]:
feature_dfs[0].sum(axis=0).sort_values().tail(10)

3327     8731.0
3398     9516.0
2357    11544.0
4882    12734.0
3448    18956.0
4073    25399.0
3321    28378.0
3723    41797.0
884     50259.0
2659    78256.0
dtype: float64

The O hop assigned each unique kind of node, (unique element, h_count, aromatic, and charge) -> a unique color. There are no false negatives, there is a 1/5000 chance of a false positive.
 
Otherwise, if you see the number 2659 assigned to different nodes in two molecules you can be very confident that those nodes are identical


In [14]:
feature_dfs[3].sum(axis=0).sort_values().tail(10)

4955     945.0
4258    1075.0
42      1080.0
4769    1096.0
1058    1386.0
285     1390.0
81      1406.0
2775    1441.0
450     2186.0
982     3922.0
dtype: float64

The 3 refers to the 3 hop neighborhoods, this means that when you see two nodes having the color 982 there is a 5000-(3+1)/5000 chance that the local 3 hop neighborhood around those nodes is identical.
 
I am using this method to convert arbitrarily sized local neighborhoods into features.
 
The 3 hop neighborhood converts each 3 hop neighborhood into a color and then a graph is encoded as a count of each of the [0, 4999] possible colors.
 


In [15]:
# for index, df in enumerate(feature_dfs):
#   df.to_csv(f'/content/drive/MyDrive/SpringBoard/Therapeutic Data Commons Projects/data/feature_hop_{index}.csv')
#   print('saved a df')

In [8]:
df = pd.read_csv('/content/TDC-DeepLearning/data/cleaned_raw_data.csv')
df

,SMILE,Target
0,CCCC(=O)Nc1ccc(N2CCN(CC)CC2)c(Cl)c1.Cl,0
1,CC(=O)N(c1ccc2oc(=O)sc2c1)S(=O)(=O)c1cccs1,1
2,Clc1ccccc1-c1nc(-c2ccccc2)n[nH]1,1
3,COc1ccccc1C(c1nnnn1C(C)(C)C)N1CCN(Cc2ccncc2)CC1,1
4,COc1ccccc1CNC(=O)Cn1nnc(-c2ccncc2)n1,1
...,...,...
12660,CCN1C(=O)[C@@H]2[C@@H](CC[C@@H]3C(=O)C=C[C@@H]...,0
12661,O=C1[C@H]2CC=C3[C@@H]([C@H](O)[C@H]4O[C@@H]4C3...,0
12662,CCN1C(=O)[C@H]2CC=C3[C@@H]([C@H](O)[C@H]4O[C@@...,1
12663,C[C@H](c1ccccc1)N1C(=O)[C@@H]2[C@@H](CC[C@@H]3...,0


In [9]:
features = pd.concat(feature_dfs[3:], axis=1).astype(np.int8)
# using the colors of hop 3 to 10
print(features.dtypes[0])
print(features.shape)


0    int8
0    int8
0    int8
0    int8
0    int8
0    int8
0    int8
dtype: object
(12665, 35000)


Each molecule is converted into a vector of the 3 hop neighborhoods appended to the 4 hop neighborhoods ... 10 hop neighborhood. That vector is then passed into the MLP.

In [22]:
from tensorflow.keras.layers import *

def compute_auprc(model, X_test, y_test):
    # https://stats.stackexchange.com/questions/157012/area-under-precision-recall-curve-auc-of-pr-curve-and-average-precision-ap
    precision, recall, thresholds = precision_recall_curve(y_test, model.predict(X_test))
    area = auc(recall, precision)
    return area

def get_optimizer():
  # https://towardsdatascience.com/simple-guide-to-hyperparameter-tuning-in-neural-networks-3fe03dad8594
	learning_rate = 0.1
	decay_rate = learning_rate / 300
	momentum = 0.8
	sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
	return sgd

def create_baseline():
  model = Sequential()
  model.add(Dense(64, activation='relu'))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer=get_optimizer(), metrics=['accuracy'])
  return model


def create_relu():
  model = Sequential()
  model.add(Dense(64, activation='relu'))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(1, activation='relu'))
  model.compile(loss='binary_crossentropy', optimizer=get_optimizer(), metrics=['accuracy'])
  return model

def create_longer_model():
  model = Sequential()
  model.add(Dense(64, activation='relu'))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer=get_optimizer(), metrics=['accuracy'])
  return model


def create_longer_thin_model():
  model = Sequential()
  model.add(Dense(32, activation='relu'))
  model.add(Dense(32, activation='relu'))
  model.add(Dense(32, activation='relu'))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer=get_optimizer(), metrics=['accuracy'])
  return model

def create_soft_max():
  model = Sequential()
  model.add(Dense(64, activation='relu'))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(1, activation='softmax'))
  model.compile(loss='binary_crossentropy', optimizer=get_optimizer(), metrics=['accuracy'])
  return model

def create_adam_optimzer():
  model = Sequential()
  model.add(Dense(64, activation='relu'))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(1, activation='softmax'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

def create_poisson_loss():
  model = Sequential()
  model.add(Dense(64, activation='relu'))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='poisson', optimizer=get_optimizer(), metrics=['accuracy'])
  return model

def create_kl_divergence():
  model = Sequential()
  model.add(Dense(64, activation='relu'))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='kl_divergence', optimizer=get_optimizer(), metrics=['accuracy'])
  return model




In [62]:
def evaluate(model_function, X, y):
  scores = []
  for i in range(5):
    model = model_function()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=i)
    for s in range(10):
      model = model_function()
      model.fit(X_train, y_train, validation_data=(X_test, y_test))
      score = compute_auprc(model, X_test, y_test)
      print(f'AUPRC : {round(score, 4)}')
      scores.append(score)
      del model
  
  print(f'Average AUPRC: {round(np.average(scores), 4)} +- {round(np.std(scores), 4)} ')



In [20]:
X, y = features.values, df['Target'].values

In [63]:
evaluate(create_baseline, X, y)

317/317 [==============================] - 1s 4ms/step - loss: 0.5597 - accuracy: 0.7033 - val_loss: 0.5034 - val_accuracy: 0.7604
AUPRC : 0.8062
317/317 [==============================] - 1s 3ms/step - loss: 0.5581 - accuracy: 0.7054 - val_loss: 0.4930 - val_accuracy: 0.7694
AUPRC : 0.8153
317/317 [==============================] - 1s 3ms/step - loss: 0.5606 - accuracy: 0.7054 - val_loss: 0.4947 - val_accuracy: 0.7671
AUPRC : 0.8188
317/317 [==============================] - 2s 5ms/step - loss: 0.5606 - accuracy: 0.7084 - val_loss: 0.5058 - val_accuracy: 0.7600
AUPRC : 0.816
317/317 [==============================] - 1s 4ms/step - loss: 0.5586 - accuracy: 0.7015 - val_loss: 0.4944 - val_accuracy: 0.7683
AUPRC : 0.8192
317/317 [==============================] - 1s 4ms/step - loss: 0.5579 - accuracy: 0.7062 - val_loss: 0.4919 - val_accuracy: 0.7627
AUPRC : 0.8139
317/317 [==============================] - 1s 3ms/step - loss: 0.5549 - accuracy: 0.7052 - val_loss: 0.5037 - val_accuracy: 0

In [64]:
evaluate(create_relu, X, y)

317/317 [==============================] - 1s 3ms/step - loss: 8.2771 - accuracy: 0.4563 - val_loss: 8.0250 - val_accuracy: 0.4737
AUPRC : 0.5341
317/317 [==============================] - 1s 4ms/step - loss: 8.2671 - accuracy: 0.4571 - val_loss: 8.0250 - val_accuracy: 0.4737
AUPRC : 0.5786
317/317 [==============================] - 1s 4ms/step - loss: 8.2822 - accuracy: 0.4563 - val_loss: 8.0250 - val_accuracy: 0.4737
AUPRC : 0.5105
317/317 [==============================] - 1s 4ms/step - loss: 8.2803 - accuracy: 0.4561 - val_loss: 8.0250 - val_accuracy: 0.4737
AUPRC : 0.525
317/317 [==============================] - 1s 4ms/step - loss: 8.2731 - accuracy: 0.4567 - val_loss: 8.0250 - val_accuracy: 0.4737
AUPRC : 0.5446
317/317 [==============================] - 1s 4ms/step - loss: 8.2755 - accuracy: 0.4563 - val_loss: 8.0250 - val_accuracy: 0.4737
AUPRC : 0.527
317/317 [==============================] - 2s 4ms/step - loss: 8.2795 - accuracy: 0.4563 - val_loss: 8.0250 - val_accuracy: 0.

In [65]:
evaluate(create_longer_model, X, y)

317/317 [==============================] - 2s 4ms/step - loss: 0.5748 - accuracy: 0.6897 - val_loss: 0.5124 - val_accuracy: 0.7521
AUPRC : 0.8003
317/317 [==============================] - 2s 4ms/step - loss: 0.5762 - accuracy: 0.6861 - val_loss: 0.5352 - val_accuracy: 0.7505
AUPRC : 0.804
317/317 [==============================] - 2s 4ms/step - loss: 0.5752 - accuracy: 0.6915 - val_loss: 0.5094 - val_accuracy: 0.7643
AUPRC : 0.8011
317/317 [==============================] - 2s 4ms/step - loss: 0.5760 - accuracy: 0.6873 - val_loss: 0.5035 - val_accuracy: 0.7509
AUPRC : 0.8108
317/317 [==============================] - 2s 4ms/step - loss: 0.5786 - accuracy: 0.6847 - val_loss: 0.5095 - val_accuracy: 0.7513
AUPRC : 0.8089
317/317 [==============================] - 2s 4ms/step - loss: 0.5788 - accuracy: 0.6836 - val_loss: 0.5192 - val_accuracy: 0.7481
AUPRC : 0.8139
317/317 [==============================] - 2s 4ms/step - loss: 0.5749 - accuracy: 0.6885 - val_loss: 0.5135 - val_accuracy: 0

In [66]:
evaluate(create_longer_thin_model,X, y)

317/317 [==============================] - 2s 4ms/step - loss: 0.5842 - accuracy: 0.6737 - val_loss: 0.5582 - val_accuracy: 0.7177
AUPRC : 0.8016
317/317 [==============================] - 2s 4ms/step - loss: 0.5757 - accuracy: 0.6873 - val_loss: 0.5038 - val_accuracy: 0.7631
AUPRC : 0.811
317/317 [==============================] - 2s 4ms/step - loss: 0.5849 - accuracy: 0.6819 - val_loss: 0.5057 - val_accuracy: 0.7663
AUPRC : 0.8058
317/317 [==============================] - 2s 4ms/step - loss: 0.5830 - accuracy: 0.6841 - val_loss: 0.5171 - val_accuracy: 0.7485
AUPRC : 0.8025
317/317 [==============================] - 2s 4ms/step - loss: 0.5754 - accuracy: 0.6916 - val_loss: 0.5049 - val_accuracy: 0.7584
AUPRC : 0.8037
317/317 [==============================] - 2s 4ms/step - loss: 0.5865 - accuracy: 0.6777 - val_loss: 0.5124 - val_accuracy: 0.7481
AUPRC : 0.7917
317/317 [==============================] - 2s 4ms/step - loss: 0.5771 - accuracy: 0.6878 - val_loss: 0.5183 - val_accuracy: 0

In [67]:
evaluate(create_soft_max, X, y)

317/317 [==============================] - 1s 3ms/step - loss: 0.5605 - accuracy: 0.4559 - val_loss: 0.4883 - val_accuracy: 0.4737
AUPRC : 0.7369
317/317 [==============================] - 1s 3ms/step - loss: 0.5590 - accuracy: 0.4559 - val_loss: 0.5007 - val_accuracy: 0.4737
AUPRC : 0.7369
317/317 [==============================] - 2s 4ms/step - loss: 0.5613 - accuracy: 0.4559 - val_loss: 0.4945 - val_accuracy: 0.4737
AUPRC : 0.7369
317/317 [==============================] - 1s 4ms/step - loss: 0.5617 - accuracy: 0.4559 - val_loss: 0.4951 - val_accuracy: 0.4737
AUPRC : 0.7369
317/317 [==============================] - 1s 3ms/step - loss: 0.5588 - accuracy: 0.4559 - val_loss: 0.4942 - val_accuracy: 0.4737
AUPRC : 0.7369
317/317 [==============================] - 1s 4ms/step - loss: 0.5632 - accuracy: 0.4559 - val_loss: 0.5070 - val_accuracy: 0.4737
AUPRC : 0.7369
317/317 [==============================] - 1s 3ms/step - loss: 0.5617 - accuracy: 0.4559 - val_loss: 0.4944 - val_accuracy: 

In [68]:
evaluate(create_adam_optimzer, X, y)

317/317 [==============================] - 2s 4ms/step - loss: 0.5684 - accuracy: 0.4559 - val_loss: 0.5157 - val_accuracy: 0.4737
AUPRC : 0.7369
317/317 [==============================] - 1s 3ms/step - loss: 0.5666 - accuracy: 0.4559 - val_loss: 0.5076 - val_accuracy: 0.4737
AUPRC : 0.7369
317/317 [==============================] - 1s 3ms/step - loss: 0.5686 - accuracy: 0.4559 - val_loss: 0.5073 - val_accuracy: 0.4737
AUPRC : 0.7369
317/317 [==============================] - 1s 3ms/step - loss: 0.5719 - accuracy: 0.4559 - val_loss: 0.5095 - val_accuracy: 0.4737
AUPRC : 0.7369
317/317 [==============================] - 2s 4ms/step - loss: 0.5701 - accuracy: 0.4559 - val_loss: 0.5040 - val_accuracy: 0.4737
AUPRC : 0.7369
317/317 [==============================] - 1s 3ms/step - loss: 0.5660 - accuracy: 0.4559 - val_loss: 0.5073 - val_accuracy: 0.4737
AUPRC : 0.7369
317/317 [==============================] - 1s 3ms/step - loss: 0.5683 - accuracy: 0.4559 - val_loss: 0.5111 - val_accuracy: 

In [69]:
evaluate(create_poisson_loss, X, y)

317/317 [==============================] - 1s 3ms/step - loss: 0.7469 - accuracy: 0.6951 - val_loss: 0.7222 - val_accuracy: 0.7659
AUPRC : 0.8097
317/317 [==============================] - 1s 3ms/step - loss: 0.7470 - accuracy: 0.6934 - val_loss: 0.7180 - val_accuracy: 0.7635
AUPRC : 0.8031
317/317 [==============================] - 1s 3ms/step - loss: 0.7432 - accuracy: 0.6983 - val_loss: 0.7261 - val_accuracy: 0.7615
AUPRC : 0.8138
317/317 [==============================] - 1s 3ms/step - loss: 0.7444 - accuracy: 0.6959 - val_loss: 0.7310 - val_accuracy: 0.7584
AUPRC : 0.8117
317/317 [==============================] - 1s 3ms/step - loss: 0.7443 - accuracy: 0.7046 - val_loss: 0.7299 - val_accuracy: 0.7580
AUPRC : 0.7954
317/317 [==============================] - 1s 3ms/step - loss: 0.7448 - accuracy: 0.6955 - val_loss: 0.7256 - val_accuracy: 0.7596
AUPRC : 0.798
317/317 [==============================] - 1s 3ms/step - loss: 0.7455 - accuracy: 0.7014 - val_loss: 0.7241 - val_accuracy: 0

In [70]:
evaluate(create_kl_divergence, X, y)

317/317 [==============================] - 1s 3ms/step - loss: 0.0082 - accuracy: 0.4561 - val_loss: 2.0764e-04 - val_accuracy: 0.4737
AUPRC : 0.5543
317/317 [==============================] - 1s 3ms/step - loss: 0.0097 - accuracy: 0.4567 - val_loss: 2.2172e-04 - val_accuracy: 0.4737
AUPRC : 0.5591
317/317 [==============================] - 1s 3ms/step - loss: 0.0094 - accuracy: 0.4557 - val_loss: 2.2342e-04 - val_accuracy: 0.4737
AUPRC : 0.5608
317/317 [==============================] - 1s 3ms/step - loss: 0.0088 - accuracy: 0.4562 - val_loss: 2.0361e-04 - val_accuracy: 0.4737
AUPRC : 0.5626
317/317 [==============================] - 1s 3ms/step - loss: 0.0094 - accuracy: 0.4553 - val_loss: 2.1510e-04 - val_accuracy: 0.4737
AUPRC : 0.551
317/317 [==============================] - 2s 5ms/step - loss: 0.0109 - accuracy: 0.4561 - val_loss: 2.5403e-04 - val_accuracy: 0.4737
AUPRC : 0.5497
317/317 [==============================] - 1s 3ms/step - loss: 0.0096 - accuracy: 0.4558 - val_loss: 

# Modeling PCA Features

In [80]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# pca_dfs = [pd.DataFrame(PCA(500).fit_transform(df)) for df in feature_dfs]
# # note: this violates cross validation rules. the Transformer is trained on the whole data

## I need to use PCA becasuse otherwise the modeling would take much to long.

In [81]:
# pca_features = pd.concat(pca_dfs, axis=1)

In [82]:
# pca_features

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499
0,-8.263492,0.963605,-0.888452,-1.243382,-2.402157,1.028729,1.171417,-0.606627,-0.208971,-0.052882,-0.380879,0.096450,0.701474,-0.216390,0.355409,0.454403,0.042462,-0.017152,-0.051059,-0.051929,-0.137322,0.036402,0.007741,0.018889,0.077123,0.315829,0.016143,-0.028928,0.863105,0.010796,-0.047515,0.004857,-0.175472,0.002358,-0.000692,0.004062,0.008048,0.001201,0.008499,0.001678,...,0.013024,-0.163675,-0.098161,0.013571,0.060841,-0.013031,-0.043281,0.044308,-0.116925,0.138890,0.078211,0.053327,0.058715,-0.060403,-0.050751,0.138405,-0.029797,-0.037245,-0.014756,-0.093928,0.154146,0.126898,-0.098928,0.017883,-0.002330,-0.022530,0.088332,-0.226134,0.015405,-0.194278,0.025890,-0.005867,-0.045948,-0.095576,0.004557,-0.007472,0.040861,0.086430,0.124055,0.130666
1,0.898335,-2.586220,2.007668,-0.334445,0.854697,-1.694698,-1.456671,-1.207205,-0.888076,-1.164089,0.499560,0.189210,-0.064412,-0.072905,0.134282,-0.409127,-0.444898,-0.449822,0.779485,0.136229,0.081331,0.054852,0.010849,-0.086942,-0.064550,0.011517,0.001365,0.061877,0.062367,0.117169,0.975778,0.002542,0.009040,-0.004726,-0.001005,-0.008120,-0.050476,-0.004626,0.020637,-0.003447,...,-0.111908,0.068251,-0.040814,0.026314,-0.042939,0.165797,-0.016452,0.013589,-0.002818,-0.134110,0.050103,0.072048,-0.165116,-0.000962,0.084185,-0.085134,-0.023837,0.052605,-0.011270,0.106052,0.020306,-0.021354,-0.071172,-0.123354,-0.039856,-0.015151,-0.038526,0.175732,0.100912,0.047202,0.076067,-0.039203,0.225685,0.069273,-0.039690,-0.107670,-0.151479,0.061278,0.076817,0.134793
2,4.493463,-2.558734,-2.572918,-0.434847,1.020059,0.149740,0.147518,0.288368,-0.047511,0.292525,0.421291,0.836481,0.755268,-0.205173,0.357187,0.363937,-0.087777,0.037143,-0.217111,-0.537706,0.119610,0.691377,-0.023645,0.113863,0.045743,0.005161,0.028205,-0.040212,-0.006089,-0.007441,0.003731,-0.000400,0.002972,-0.000908,0.000561,0.000258,-0.000871,-0.007174,-0.001034,-0.000342,...,-0.002293,0.031711,0.060542,0.067907,0.061881,0.048595,0.024664,0.116906,0.001380,-0.031186,0.131435,0.066869,-0.002066,-0.068508,0.041695,0.047336,-0.038962,0.046600,0.043309,0.066463,-0.004767,-0.047208,0.139329,0.015933,-0.120411,0.007890,-0.026174,-0.064839,-0.095108,0.071367,-0.015347,0.091711,0.005876,0.100599,-0.040557,0.011242,-0.121101,0.131846,-0.021940,-0.019103
3,0.378278,2.553361,-0.331753,-0.703023,-1.579869,3.296105,-0.717182,0.935663,-2.470335,-0.864609,0.288323,-1.294303,-0.613980,0.415188,0.071927,0.595980,0.287175,0.056925,0.056952,0.020765,0.007718,0.141895,0.047099,0.093296,0.026593,0.003976,0.027559,-0.046184,-0.020866,-0.005153,0.018571,0.010600,0.002055,-0.005012,-0.011922,-0.000157,0.000736,0.001278,0.004375,-0.000089,...,-0.032879,0.111451,0.061055,-0.043186,0.044480,0.042353,0.236042,0.089337,0.068823,-0.333076,-0.024625,0.280530,-0.018647,0.066760,0.362452,-0.001539,0.194706,0.059396,0.066434,0.045134,-0.276469,-0.014029,0.076696,0.005628,-0.111670,0.000378,0.189982,0.217731,-0.041345,-0.043073,0.041868,-0.030670,-0.127163,0.091768,0.011786,-0.239398,-0.116666,-0.109355,0.102852,-0.108537
4,1.681549,-0.250663,-0.265615,-1.825691,-0.529616,0.103072,-0.948659,0.407218,-1.687717,0.327295,-0.084717,-1.151620,-0.431533,0.338653,-0.331170,0.530773,0.035817,-0.036614,-0.050632,-0.065505,0.122929,0.000269,0.054743,-0.026335,-0.026221,0.012281,-0.003416,-0.020844,-0.003432,0.014751,0.000870,0.004584,-0.006200,-0.008537,-0.008863,-0.002276,0.001098,0.001891,0.001588,0.000322,...,0.008487,-0.014163,0.107424,-0.077403,-0.028502,0.040984,0.083651,0.021333,0.129179,0.089335,-0.046449,-0.070107,-0.154608,0.193414,0.058730,-0.123635,-0.104997,0.086306,-0.036501,-0.044235,0.153661,-0.116417,-0.078494,-0.073026,0.096064,-0.042397,0.115258,0.078602,-0.116903,-0.034365,0.0

In [107]:
from lightgbm import LGBMRegressor
def evaluate_lbg(model_function, X, y):
  scores = []
  for i in range(10): # smaller sample since it takes much longer
    model = LGBMRegressor(silent=False)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=i)
    model.fit(X_train, y_train)
    score = compute_auprc(model, X_test, y_test)
    print(f'AUPRC : {round(score, 4)}')
    scores.append(score)
    del model

  print(f'Average AUPRC: {round(np.average(scores), 4)} +- {round(np.std(scores), 4)} ')

In [108]:
%%time
evaluate_lbg(LGBMRegressor, pca_features.values, y) # looks very promising

AUPRC : 0.8589
AUPRC : 0.8289
AUPRC : 0.8433
AUPRC : 0.8282
AUPRC : 0.8376
AUPRC : 0.8405
AUPRC : 0.8307
AUPRC : 0.8266
AUPRC : 0.8402
AUPRC : 0.8517
Average AUPRC: 0.8387 +- 0.0101 
CPU times: user 30min 17s, sys: 7.5 s, total: 30min 25s
Wall time: 15min 39s


# PCA not violating cross validation

In [136]:
features = features.astype(np.int16)

In [19]:
from sklearn.decomposition import PCA

y = df['Target'].values

def get_pca_transformed_split_data(features, feature_df, seed):

  np.random.seed(seed)
  test_indices = np.random.choice(features.index, int(features.shape[0]*.2), replace=False) # set aside a 20% test set.
  test_index_mask = np.array([i in test_indices for i in features.index])
  train_index_mask = ~test_index_mask

  transformed_train_features = []
  transformed_test_features = []
  for df in feature_dfs:
    pca_model = PCA(500).fit(df[train_index_mask])
    print('fit pca model')

    transformed_train_df = pd.DataFrame(pca_model.transform(df[train_index_mask]))
    transformed_test_df = pd.DataFrame(pca_model.transform(df[test_index_mask]))

    transformed_train_features.append(transformed_train_df)
    transformed_test_features.append(transformed_test_df)


  X_train = pd.concat(transformed_train_features, axis=1).values
  y_train = y[train_index_mask]


  X_test = pd.concat(transformed_test_features, axis=1).values
  y_test = y[test_index_mask]

  return X_train, X_test, y_train, y_test


X_train, X_test, y_train, y_test = get_pca_transformed_split_data(features, feature_dfs, 55)


fit pca model
fit pca model
fit pca model
fit pca model
fit pca model
fit pca model
fit pca model
fit pca model
fit pca model
fit pca model


In [20]:
X_train.shape, y_train.shape

((10132, 5000), (10132,))

In [21]:
from lightgbm import LGBMRegressor
model = LGBMRegressor()
model.fit(X_train, y_train)
print('fit lgb model')



fit lgb model


NameError: ignored

In [23]:
score = compute_auprc(model, X_test, y_test)
print(f'AUPRC : {round(score, 4)}')

AUPRC : 0.8531


In [25]:
%%time
scores = []
for i in range(10):
  X_train, X_test, y_train, y_test = get_pca_transformed_split_data(features, feature_dfs, i)
  model = LGBMRegressor()
  model.fit(X_train, y_train)
  print('fit lgb model')
  score = compute_auprc(model, X_test, y_test)
  print(f'AUPRC : {round(score, 4)}')
  scores.append(score)

print(f'Average AUPRC: {round(np.average(scores), 4)} +- {round(np.std(scores), 4)} ')

fit pca model
fit pca model
fit pca model
fit pca model
fit pca model
fit pca model
fit pca model
fit pca model
fit pca model
fit pca model
fit lgb model
AUPRC : 0.8499
fit pca model
fit pca model
fit pca model
fit pca model
fit pca model
fit pca model
fit pca model
fit pca model
fit pca model
fit pca model
fit lgb model
AUPRC : 0.8307
fit pca model
fit pca model
fit pca model
fit pca model
fit pca model
fit pca model
fit pca model
fit pca model
fit pca model
fit pca model
fit lgb model
AUPRC : 0.8408
fit pca model
fit pca model
fit pca model
fit pca model
fit pca model
fit pca model
fit pca model
fit pca model
fit pca model
fit pca model
fit lgb model
AUPRC : 0.8433
fit pca model
fit pca model
fit pca model
fit pca model
fit pca model
fit pca model
fit pca model
fit pca model
fit pca model
fit pca model
fit lgb model
AUPRC : 0.8458
fit pca model
fit pca model
fit pca model
fit pca model
fit pca model
fit pca model
fit pca model
fit pca model
fit pca model
fit pca model
fit lgb model
A

# current best model


ColorRefinement (hops 0-10) -> PCA (500 features) -> 5000 long PCA transformed features of the color refinements -> out of the box LGBM 


Average AUPRC: 0.8375 +- 0.0093.

A significant improvement on the current https://tdcommons.ai/benchmark/admet_group/10cyp2c9i/

CNN (DeepPurpose)	Kexin Huang	GitHub, Paper	226,625	0.713 ± 0.006
